In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from datetime import datetime

from data_helpers.syntax_graph import SyntaxGraph
from data_helpers.tasak_reader import TasakReader

# functions for creating database and collecting collocations
from collect_functions_1 import *


# Tabel 2 andmete kogumine

Erinevus võrreldes tabeliga 1: tegemisel ei arvestata objektil ja subjektil käänet. Et nt obj nom, gen ja par eri sageduste asemel on ainult üks suur * sagedus? 

In [7]:
%%time

file_name = 'data/tasak.vert'

my_reader = TasakReader(
   file_name = file_name
)

CPU times: user 21 µs, sys: 14 µs, total: 35 µs
Wall time: 37.9 µs


In [8]:
%%time

TYPE = 'obl_verb_subj_obj_nocase'
TABLENAME = f'{TYPE}'
BATCH_SIZE = 100000

date_time = datetime.now().strftime("%Y%m%d-%H%M%S")
db_file_name = f"tasak_{TYPE}_collocations_{date_time}.db"

my_sqlite_db = DbMethods(db_file_name=db_file_name, table1_name=TYPE, table2_name=TYPE+'_examples')
my_sqlite_db.prep_coll_db()

no_subj_obj_case = True

# hoitakse algusest lõpuni mälus
verb_global_stat = {}

# kollokatsioonid, tühjendatakse peale igat salvestamist
collocations = {}
count = 0
for collection_id, graph in my_reader.get_sentences():
    collection_id = int(collection_id)
    count += 1
    collocations, verb_global_stat, = extract_something(graph, collection_id, collocations, verb_global_stat, no_subj_obj_case )
    if not collection_id == 0 and not collection_id % BATCH_SIZE:
        my_sqlite_db.save_coll_to_db(collocations, collection_id)
        collocations = {}
    
    
# saving last batch
my_sqlite_db.save_coll_to_db(collocations, collection_id)
collocations = {}
my_sqlite_db.index_fields()

df = verb_stat_to_df(verb_global_stat, f'tasak_verb_kids_{date_time}.csv')
df.to_sql(name='verbs_global_stat', con=my_sqlite_db._connection)

print('done.')

data/tasak.vert


TSV lines:   1%|          | 214580/20058039 [00:00<01:29, 221524.16it/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 200000


TSV lines:   6%|▋         | 1290741/20058039 [00:05<01:51, 167895.66it/s]

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 300000


TSV lines:  24%|██▎       | 4730530/20058039 [00:22<02:55, 87509.48it/s] 

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 6900000


TSV lines:  29%|██▉       | 5805931/20058039 [00:27<02:22, 99893.79it/s] 

andmebaasi salvestatud kollokatsioonid kollektsioonidest: 0 - 7100000


TSV lines:  35%|███▌      | 7053681/20058039 [00:33<00:56, 228712.39it/s]